In [135]:
import pandas as pd
import numpy as np

In [2]:
def load_data(path):
    '''
    Given a path (str) to a csv returns a pandas dataframe
    '''
    df = pd.read_csv(path)
    return df

In [3]:
carsDf = load_data("./cars.csv")
carsDf.head()

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,True,False,True,True,False,False,False,False,True,7


In [20]:
carsDf.shape[0] # number of rows of the data

38531

In [4]:
# Get info about the data
carsDf.dtypes

manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas          bool
engine_type           object
engine_capacity      float64
body_type             object
has_warranty            bool
state                 object
drivetrain            object
price_usd            float64
is_exchangeable         bool
location_region       object
number_of_photos       int64
up_counter             int64
feature_0               bool
feature_1               bool
feature_2               bool
feature_3               bool
feature_4               bool
feature_5               bool
feature_6               bool
feature_7               bool
feature_8               bool
feature_9               bool
duration_listed        int64
dtype: object

## Convert all Boolean Categorical (True and False) to binary (0 or 1)

In [165]:
def bool_to_bin(df):
    '''
    Converts all True or False Values into binary (0 or 1)
    Returns a new df with the updates
    '''
    df_updated = df.copy(deep=True) # make a copy 
    for col in df.columns:
        if df[col].dtype == bool:
            df_updated[col] = df[col].astype(int)
    return df_updated
    

In [6]:
carsDf_bin = bool_to_bin(carsDf)
# check results visually
carsDf_bin.dtypes

manufacturer_name     object
model_name            object
transmission          object
color                 object
odometer_value         int64
year_produced          int64
engine_fuel           object
engine_has_gas         int64
engine_type           object
engine_capacity      float64
body_type             object
has_warranty           int64
state                 object
drivetrain            object
price_usd            float64
is_exchangeable        int64
location_region       object
number_of_photos       int64
up_counter             int64
feature_0              int64
feature_1              int64
feature_2              int64
feature_3              int64
feature_4              int64
feature_5              int64
feature_6              int64
feature_7              int64
feature_8              int64
feature_9              int64
duration_listed        int64
dtype: object

In [7]:
# verify no bool columns exists
for col in carsDf_bin:
    assert(carsDf_bin[col].dtype != bool) 

In [8]:
# if everything is valid update original df
carsDf = carsDf_bin.copy(deep=True)

## Remove Records with Missing Data

In [16]:
def remove_missing(df):
    '''
    Removes all records that contain missing values
    This will include categorical and numeric data
    A new df is returned rather than updated the one passed in
    '''
    df_copy = df.copy(deep=True)
    df_copy.dropna() # removes all records with nan (missing values)
    return df_copy

In [21]:
carsDf_drop_missing = remove_missing(carsDf)
carsDf.shape[0], carsDf_drop_missing.shape[0] # no missing values found. Matches documentation description of each feature

(38531, 38531)

In [46]:
# if everything is valid update original df
carsDf = carsDf_drop_missing.copy(deep=True)

# Update String Categorical Data 

In [70]:
# Find all remaining columns
str_cols = []
threshold_mapping = {}
for col in carsDf:
    if carsDf[col].dtypes == object:
        str_cols.append(col)
print(str_cols)

['manufacturer_name', 'model_name', 'transmission', 'color', 'engine_fuel', 'engine_type', 'body_type', 'state', 'drivetrain', 'location_region']


## Combine Levels
- Convert Low Frequency Column Values to Other 

In [83]:
def other_columns_describe(df, col, printInd):
    '''
    Function makes a count data frame of unique values of a column 
    and prints the results in order if printInd is set to True
    '''
    # Group by the unique values and provide a count
    countsDf = df.groupby([col], dropna=False).size().reset_index(name='counts')
    countsDf["rel_freq"] = (countsDf["counts"] / countsDf["counts"].sum() ) * 100 # make a relative frequency
    
    # figure out the threshold
    if printInd:
        print(countsDf.sort_values(by=["rel_freq"], ascending = True))
    return countsDf
    

In [167]:
def other_columns(df, col, threshold):
    '''
    Updates columns with frequency below a threshold to be labeled as other
    New df returned
    '''
    df_copy = df.copy(deep=True) # make a copy
    # Group by the unique values and provide a count
    countDf= other_columns_describe(df, col, False)
    
    # get all col values below the threshold
    countDf = countDf.loc[countDf["rel_freq"] < threshold]
    other_values = countDf[col].tolist()
    
    # update those columns to be other type
    df_copy.loc[(df_copy[col].isin(other_values)), col] = "other"
    
    return df_copy

In [84]:
# figure out a threshold for column
count_1 = other_columns_describe(carsDf, str_cols[0], True)

   manufacturer_name  counts   rel_freq
51               ЗАЗ      42   0.109003
35           Pontiac      42   0.109003
5           Cadillac      43   0.111598
4              Buick      47   0.121980
27             Lifan      47   0.121980
20            Jaguar      53   0.137552
52           Москвич      55   0.142742
6              Chery      58   0.150528
10             Dacia      59   0.153123
36           Porsche      61   0.158314
0              Acura      66   0.171291
30              Mini      68   0.176481
15             Geely      71   0.184267
48             other      72   0.186863
53               УАЗ      74   0.192053
42         SsangYong      79   0.205030
24            Lancia      92   0.238769
21              Jeep     107   0.277698
39              Saab     108   0.280294
19             Iveco     139   0.360748
23              LADA     146   0.378916
18          Infiniti     162   0.420441
25        Land Rover     184   0.477538
50               ГАЗ     200   0.519063


In [75]:
threshold_mapping[str_cols[0]] =.1

In [76]:
carsDf_1 = other_columns(carsDf, str_cols[0], threshold_mapping[str_cols[0]])

In [77]:
assert (carsDf_1[str_cols[0]].unique().size ) == (carsDf[str_cols[0]].unique().size -1 ) # should be one less value 
# two values converted to other

In [78]:
# if everything is valid update original df
carsDf = carsDf_1.copy(deep=True)

In [85]:
# Next Columns
# figure out a threshold for column
count_2 = other_columns_describe(carsDf, str_cols[1], True)

    model_name  counts  rel_freq
893        SLX       1  0.002595
979        TLX       1  0.002595
139        410       1  0.002595
540      GL420       1  0.002595
142  423 Kombi       1  0.002595
..         ...     ...       ...
734     Mondeo     637  1.653214
228         A6     687  1.782980
569       Golf     707  1.834886
258      Astra     751  1.949080
778     Passat    1423  3.693130

[1118 rows x 3 columns]


In [91]:
threshold_mapping[str_cols[1]] =.01
carsDf_2 = other_columns(carsDf, str_cols[1], threshold_mapping[str_cols[1]])
carsDf_2[str_cols[1]].unique().size , carsDf[str_cols[1]].unique().size

(760, 1118)

In [92]:
# if everything is valid update original df
carsDf = carsDf_2.copy(deep=True)

In [94]:
# next column
# figure out a threshold for column
count_3 = other_columns_describe(carsDf, str_cols[2], True) # do not convert to other

  transmission  counts   rel_freq
0    automatic   12898  33.474345
1   mechanical   25633  66.525655


In [95]:
threshold_mapping[str_cols[2]] =.01
carsDf_3 = other_columns(carsDf, str_cols[2], threshold_mapping[str_cols[2]])
carsDf_3[str_cols[2]].unique().size , carsDf[str_cols[2]].unique().size # no change, no need to update

(2, 2)

In [98]:
# next column
# figure out a threshold for column
count_4 = other_columns_describe(carsDf, str_cols[3], True) # do not convert to other

     color  counts   rel_freq
5   orange     184   0.477538
11  yellow     303   0.786380
9   violet     463   1.201630
2    brown     886   2.299447
6    other    2688   6.976201
3    green    2760   7.163064
7      red    2925   7.591290
4     grey    3751   9.735019
10   white    4212  10.931458
1     blue    5802  15.058005
8   silver    6852  17.783084
0    black    7705  19.996886


In [102]:
threshold_mapping[str_cols[3]] = 1
carsDf_4 = other_columns(carsDf, str_cols[3], threshold_mapping[str_cols[3]])
carsDf_4[str_cols[3]].unique().size , carsDf[str_cols[3]].unique().size 

(10, 12)

In [103]:
# if everything is valid update original df
carsDf = carsDf_4.copy(deep=True)

In [104]:
# next column
# figure out a threshold for column
count_5 = other_columns_describe(carsDf, str_cols[4], True) # do not convert to other

     engine_fuel  counts   rel_freq
4  hybrid-diesel       2   0.005191
1       electric      10   0.025953
5  hybrid-petrol     235   0.609899
2            gas    1347   3.495886
0         diesel   12872  33.406867
3       gasoline   24065  62.456204


In [105]:
threshold_mapping[str_cols[4]] = .5
carsDf_5 = other_columns(carsDf, str_cols[4], threshold_mapping[str_cols[4]])
carsDf_5[str_cols[4]].unique().size , carsDf[str_cols[4]].unique().size 

(4, 6)

In [106]:
# if everything is valid update original df
carsDf = carsDf_5.copy(deep=True)

In [111]:
# next column
# figure out a threshold for column
count_6 = other_columns_describe(carsDf, str_cols[5], True) # do not convert to other

  engine_type  counts   rel_freq
1    electric      10   0.025953
0      diesel   12874  33.412058
2    gasoline   25647  66.561989


In [117]:
threshold_mapping[str_cols[5]] = .001
carsDf_6 = other_columns(carsDf, str_cols[5], threshold_mapping[str_cols[5]])
carsDf_6[str_cols[5]].unique().size , carsDf[str_cols[5]].unique().size # no change don't update

(3, 3)

In [119]:
# next column
# figure out a threshold for column
count_7 = other_columns_describe(carsDf, str_cols[6], True) # do not convert to other

    body_type  counts   rel_freq
4   limousine      12   0.031144
0   cabriolet      75   0.194648
7      pickup     129   0.334795
3    liftback     552   1.432613
1       coupe     652   1.692144
11        van     808   2.097013
5     minibus    1369   3.552983
6     minivan    3608   9.363889
9         suv    5164  13.402196
10  universal    5507  14.292388
2   hatchback    7644  19.838572
8       sedan   13011  33.767616


In [120]:
threshold_mapping[str_cols[6]] = 1
carsDf_7 = other_columns(carsDf, str_cols[6], threshold_mapping[str_cols[6]])
carsDf_7[str_cols[6]].unique().size , carsDf[str_cols[6]].unique().size # no change don't update

(10, 12)

In [121]:
# next column
# figure out a threshold for column
count_8 = other_columns_describe(carsDf, str_cols[7], True) # do not convert to other

       state  counts   rel_freq
0  emergency     370   0.960266
1        new     438   1.136747
2      owned   37723  97.902987


In [122]:
threshold_mapping[str_cols[7]] = .1
carsDf_8 = other_columns(carsDf, str_cols[7], threshold_mapping[str_cols[7]])
carsDf_8[str_cols[7]].unique().size , carsDf[str_cols[7]].unique().size # no change don't update

(3, 3)

In [123]:
# next column
# figure out a threshold for column
count_9 = other_columns_describe(carsDf, str_cols[8], True) # do not convert to other

  drivetrain  counts   rel_freq
0        all    5387  13.980950
2       rear    5398  14.009499
1      front   27746  72.009551


In [124]:
threshold_mapping[str_cols[8]] = 1
carsDf_9 = other_columns(carsDf, str_cols[8], threshold_mapping[str_cols[8]])
carsDf_9[str_cols[8]].unique().size , carsDf[str_cols[8]].unique().size # no change don't update

(3, 3)

In [126]:
# next column
# figure out a threshold for column
count_10 = other_columns_describe(carsDf, str_cols[9], True) # do not convert to other

    location_region  counts   rel_freq
3  Гродненская обл.    2485   6.449352
5  Могилевская обл.    2678   6.950248
0    Брестская обл.    2991   7.762581
1    Витебская обл.    3007   7.804106
2   Гомельская обл.    3140   8.149282
4      Минская обл.   24230  62.884431


In [127]:
threshold_mapping[str_cols[9]] = 1
carsDf_10 = other_columns(carsDf, str_cols[9], threshold_mapping[str_cols[9]])
carsDf_10[str_cols[9]].unique().size , carsDf[str_cols[9]].unique().size # no change don't update

(6, 6)

# Perform One Hot Encoding

In [156]:
def one_hot(df, col):
    '''
    Performs one hot encoding on a single column
    A new dataframe is returned
    One hot encoding is a list of binary values
    '''
    df_c = df.copy(deep=True)
    uniqueV = df[col].unique().tolist()
    cnt = len(uniqueV)
    one_hot_encoding = []
    
    for i, rw in df_c.iterrows():
        index = uniqueV.index(rw[col]) # index in unique list
        encoding = [ 1 if index ==i else 0 for i in range(cnt)] # set 1 if index else 0
        one_hot_encoding.append(encoding)
    df_c[col] = one_hot_encoding
    return df_c
        

In [151]:
df_1 = one_hot(carsDf, "drivetrain")

In [154]:
# compare some of the results
df_1["drivetrain"].head()

0    [1, 0, 0]
1    [1, 0, 0]
2    [1, 0, 0]
3    [1, 0, 0]
4    [1, 0, 0]
Name: drivetrain, dtype: object

In [155]:
carsDf["drivetrain"].head()

0    all
1    all
2    all
3    all
4    all
Name: drivetrain, dtype: object

In [157]:
# apply one hot to all columns
for col in str_cols:
    df_col = one_hot(carsDf, col)
    carsDf = df_col.copy(deep=True)
    

In [161]:
carsDf.head() # verify the data is as expected now

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",190000,2010,"[1, 0, 0, 0]",0,"[1, 0, 0]",2.5,...,1,1,1,0,1,0,1,1,1,16
1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",290000,2002,"[1, 0, 0, 0]",0,"[1, 0, 0]",3.0,...,1,0,0,1,1,0,0,0,1,83
2,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",402000,2001,"[1, 0, 0, 0]",0,"[1, 0, 0]",2.5,...,1,0,0,0,0,0,0,1,1,151
3,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",10000,1999,"[1, 0, 0, 0]",0,"[1, 0, 0]",3.0,...,0,0,0,0,0,0,0,0,0,86
4,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",280000,2001,"[1, 0, 0, 0]",0,"[1, 0, 0]",2.5,...,1,0,1,1,0,0,0,0,1,7


In [164]:
# confirm no more string values
for i in range(carsDf.shape[0]):
    for col in str_cols:
        assert isinstance(carsDf.iloc[i][col],list) # should be a list/encoding and not string anymor

In [178]:
def prep_for_training(df, label):
    '''
    To feed data into a ML model like NN,
    we need the data split into a features and label
    and the data should be stored in numpy arrays instead of pandas dfs
    '''
    cols = df.columns.tolist()
    cols.remove(label)
    
    y = df[label].to_numpy()
    x = df[cols].to_numpy()
    return x, y

In [180]:
# convert data to be feed into a ML model
x, y = prep_for_training(carsDf,"price_usd")